# Importing Section

In [1]:
import findspark
findspark.init("C:\Hadoop\spark-3.0.0-bin-hadoop3.2")
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Training').getOrCreate()

In [2]:
from pyspark.sql.functions import when,udf,lit,split

In [3]:
from pyspark.sql.types import BooleanType

In [4]:
import re

In [5]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

# Reading CSV Section

In [6]:
locationDF = spark.read.csv("dl/location.csv", header=True, inferSchema=True)

In [7]:
provinceDF = spark.read.csv("dl/province.csv", header=True, inferSchema=True)

In [8]:
countryDF = spark.read.csv("dl/country.csv", header=True, inferSchema=True)

In [9]:
carDF = spark.read.csv("dl/car.csv", header=True, inferSchema=True)

In [10]:
customerDF = spark.read.csv("dl/customer.csv", header=True, inferSchema=True)

In [11]:
customer_locationDF = spark.read.csv(path=["dl/customer_location.csv",
                                           "dl/customer_location2.csv",
                                           "dl/customer_location3.csv",
                                           "dl/customer_location4.csv"], header=True, inferSchema=True)

In [12]:
timeDF = spark.read.csv("dl/time.csv", header=True, inferSchema=True)

# DF View

In [13]:
locationDF.show()

+------+-------------------+-------+---------+-------+----------+
|loc_id|            address|   city|postal_cd|prov_nm|country_nm|
+------+-------------------+-------+---------+-------+----------+
|     1| 10 Yonge st apt 50|Toronto|  M51 3W2|Ontario|    Canada|
|     2|      3500 Bloor St|Toronto|  M6E 2I3|Ontario|    Canada|
|     3|     200 College St|Toronto|  M4B 2B9|Ontario|    Canada|
|     4|20 Yonge St apt 413|Toronto|  M51 3W2|Ontario|    Canada|
|     5|      2600 Keele St|Toronto|  M2O 2P3|Ontario|    Canada|
|     6|        10 St Clair|Toronto|  M3Q 4I2|Ontario|    Canada|
|     7|   20 Sherbourne St|Toronto|  M2O BAD|Ontario|    Canada|
|     8|      40 College St|Toronto|  M4B 2B9|Ontario|    Canada|
|     9|      6001 Bloor St|Toronto|  M6E 2I3|Ontario|    Canada|
|    10|       501 Queen St|Toronto|  M8K 1J3|Ontario|    Canada|
|    11|       1004 King St|Toronto|  M9J 2K4|Ontario|    Canada|
|    12|  201 Wellington St|Toronto|  M2E 2J3|Ontario|    Canada|
|    13|  

In [14]:
provinceDF.show()

+-----------+----------+-------------+----------------+
|province_id|country_id|province_code|   province_name|
+-----------+----------+-------------+----------------+
|          1|         2|           AL|         Alabama|
|          2|         2|           AK|          Alaska|
|          3|         1|           AB|         Alberta|
|          4|         2|           AZ|         Arizona|
|          5|         2|           AR|        Arkansas|
|          6|         1|           BC|British Columbia|
|          7|         2|           CA|      California|
|          8|         2|           CO|        Colorado|
|          9|         2|           CT|     Connecticut|
|         10|         2|           DE|        Delaware|
|         11|         2|           FL|         Florida|
|         12|         2|           GA|         Georgia|
|         13|         2|           HI|          Hawaii|
|         14|         2|           ID|           Idaho|
|         15|         2|           IL|        Il

In [15]:
countryDF.show()

+----------+--------------------+--------------+
|country_id|        country_name|country_iso_cd|
+----------+--------------------+--------------+
|         1|              Canada|           CAD|
|         2|United States of ...|           USD|
+----------+--------------------+--------------+



In [16]:
carDF.show()

+------+---------+---------+----+
|car_id|car_brand|car_model|year|
+------+---------+---------+----+
|     1|   Volv 0|     XC40|2019|
|     2|    Volvo|     XC60|201A|
|     3|    Volvo|      S90|2020|
|     4|    Volvo|      S60|2019|
|     5|    Volvo|    P1800|1970|
|     6|     Audi|       TT|2019|
|     7|     Audi|       A3|2020|
|     8|     Audi|       A4|2015|
|     9|     Audi|       A6|2020|
|    10|      BMW|      330|2019|
|    11|      BMW|      340|2010|
|    12|      BMW|      540|2020|
|    13|      BMW|       M3|2019|
|    14|      BMW|       XX|2020|
+------+---------+---------+----+



In [17]:
customer_locationDF.show()

+-------+------+-----------+
|Cust_id|loc_id|     chg_dt|
+-------+------+-----------+
|      1|     1| 2019-01-21|
|      2|     2| 2020-01-19|
|      3|     3| 2019-02-27|
|      4|     4| 2019-09-26|
|      5|     5| 2019-05-07|
|      6|     6| 2019-05-17|
|      7|     7| 2020-03-26|
|      8|     8| 2019-10-01|
|      9|     9| 2019-10-03|
|     10|    10| 2020-01-12|
|     11|    11| 2019-06-19|
|     12|    12| 2019-07-27|
|     13|    13| 2019-05-03|
|     14|    14| 2019-05-31|
|     15|    15| 2020-03-22|
|     16|    16|2020 Jan 12|
|     17|    17|2019-Rea-10|
|     18|    18| 2019-01-27|
|     19|    19| 2019-06-25|
|     20|    20| 2019-05-29|
+-------+------+-----------+
only showing top 20 rows



In [18]:
timeDF.show()

+-------+----------+---+-----+----+
|time_id|   time_dt|day|month|year|
+-------+----------+---+-----+----+
|      1|2019-01-01|  1|    1|2019|
|      2|2019-01-02|  2|    1|2019|
|      3|2019-01-03|  3|    1|2019|
|      4|2019-01-04|  4|    1|2019|
|      5|2019-01-05|  5|    1|2019|
|      6|2019-01-06|  6|    1|2019|
|      7|2019-01-07|  7|    1|2019|
|      8|2019-01-08|  8|    1|2019|
|      9|2019-01-09|  9|    1|2019|
|     10|2019-01-10| 10|    1|2019|
|     11|2019-01-11| 11|    1|2019|
|     12|2019-01-12| 12|    1|2019|
|     13|2019-01-13| 13|    1|2019|
|     14|2019-01-14| 14|    1|2019|
|     15|2019-01-15| 15|    1|2019|
|     16|2019-01-16| 16|    1|2019|
|     17|2019-01-17| 17|    1|2019|
|     18|2019-01-18| 18|    1|2019|
|     19|2019-01-19| 19|    1|2019|
|     20|2019-01-20| 20|    1|2019|
+-------+----------+---+-----+----+
only showing top 20 rows



# Data Quality Rules

### Rule 1

In [19]:
case1 = locationDF.postal_cd.rlike('[a-zA-Z]\d[a-zA-Z]\s\d[a-zA-Z]\d$|\d\d\d\d\d$')

In [20]:
locationDF = locationDF.withColumn('rule1', case1)

In [21]:
def checkRegex(name,df,column,exp):
    temp = df.withColumnRenamed(column,'ptemp')
    result = temp.withColumn(name, temp.ptemp.rlike(exp)).drop('ptemp')
    
    return result

In [22]:
locationDF = checkRegex('rule10',locationDF,'postal_cd','[a-zA-Z]\d[a-zA-Z]\s\d[a-zA-Z]\d$|\d\d\d\d\d$')

In [23]:
locationDF.show()

+------+-------------------+-------+-------+----------+-----+------+
|loc_id|            address|   city|prov_nm|country_nm|rule1|rule10|
+------+-------------------+-------+-------+----------+-----+------+
|     1| 10 Yonge st apt 50|Toronto|Ontario|    Canada|false| false|
|     2|      3500 Bloor St|Toronto|Ontario|    Canada| true|  true|
|     3|     200 College St|Toronto|Ontario|    Canada| true|  true|
|     4|20 Yonge St apt 413|Toronto|Ontario|    Canada|false| false|
|     5|      2600 Keele St|Toronto|Ontario|    Canada| true|  true|
|     6|        10 St Clair|Toronto|Ontario|    Canada| true|  true|
|     7|   20 Sherbourne St|Toronto|Ontario|    Canada|false| false|
|     8|      40 College St|Toronto|Ontario|    Canada| true|  true|
|     9|      6001 Bloor St|Toronto|Ontario|    Canada| true|  true|
|    10|       501 Queen St|Toronto|Ontario|    Canada| true|  true|
|    11|       1004 King St|Toronto|Ontario|    Canada| true|  true|
|    12|  201 Wellington St|Toront

### Rule 2

In [24]:
case2 = locationDF.join(provinceDF.select('province_name'), locationDF.prov_nm == provinceDF.province_name, 'left')

In [25]:
locationDF = case2.withColumn('rule2', when(case2.province_name.isNull(),False).otherwise(True)).drop('province_name')

In [26]:
def checkLink(name,df1,column1,df2,column2):
    
    # Rename the column in the second dataframe to avoid ambiguity
    temp = df2.withColumnRenamed(column2,'ptemp').select('ptemp')
    
    # Execute the left join to check links
    case = df1.join(temp, column1 == temp.ptemp, 'left')
    
    # Create the column with true or false argument according to null values in left join
    result = case.withColumn(name, when(case.ptemp.isNull(),False).otherwise(True)).drop('ptemp')
    
    return result

In [29]:
locationDF = checkLink('rule20',locationDF,locationDF.prov_nm,provinceDF,'province_name')

In [30]:
locationDF.show()

+------+-------------------+-------+-------+----------+-----+------+-----+------+
|loc_id|            address|   city|prov_nm|country_nm|rule1|rule10|rule2|rule20|
+------+-------------------+-------+-------+----------+-----+------+-----+------+
|     1| 10 Yonge st apt 50|Toronto|Ontario|    Canada|false| false| true|  true|
|     2|      3500 Bloor St|Toronto|Ontario|    Canada| true|  true| true|  true|
|     3|     200 College St|Toronto|Ontario|    Canada| true|  true| true|  true|
|     4|20 Yonge St apt 413|Toronto|Ontario|    Canada|false| false| true|  true|
|     5|      2600 Keele St|Toronto|Ontario|    Canada| true|  true| true|  true|
|     6|        10 St Clair|Toronto|Ontario|    Canada| true|  true| true|  true|
|     7|   20 Sherbourne St|Toronto|Ontario|    Canada|false| false| true|  true|
|     8|      40 College St|Toronto|Ontario|    Canada| true|  true| true|  true|
|     9|      6001 Bloor St|Toronto|Ontario|    Canada| true|  true| true|  true|
|    10|       5

### Rule 3

In [31]:
case3 = locationDF.join(countryDF.select('country_name'), locationDF.country_nm == countryDF.country_name, 'left')

In [32]:
locationDF = case3.withColumn('rule3', when(case3.country_name.isNull(),False).otherwise(True)).drop('country_name')

In [33]:
locationDF = checkLink('rule30',locationDF,locationDF.country_nm,countryDF,'country_name')

In [34]:
locationDF.show()

+------+-------------------+-------+-------+----------+-----+------+-----+------+-----+------+
|loc_id|            address|   city|prov_nm|country_nm|rule1|rule10|rule2|rule20|rule3|rule30|
+------+-------------------+-------+-------+----------+-----+------+-----+------+-----+------+
|     1| 10 Yonge st apt 50|Toronto|Ontario|    Canada|false| false| true|  true| true|  true|
|     2|      3500 Bloor St|Toronto|Ontario|    Canada| true|  true| true|  true| true|  true|
|     3|     200 College St|Toronto|Ontario|    Canada| true|  true| true|  true| true|  true|
|     4|20 Yonge St apt 413|Toronto|Ontario|    Canada|false| false| true|  true| true|  true|
|     5|      2600 Keele St|Toronto|Ontario|    Canada| true|  true| true|  true| true|  true|
|     6|        10 St Clair|Toronto|Ontario|    Canada| true|  true| true|  true| true|  true|
|     7|   20 Sherbourne St|Toronto|Ontario|    Canada|false| false| true|  true| true|  true|
|     8|      40 College St|Toronto|Ontario|    Ca

### Rule 4

#### Rule 4_1

In [35]:
case4_1 = carDF.car_brand.rlike('^[a-zA-Z]+$')

In [36]:
carDF = carDF.withColumn('rule4_1', case4_1)

In [61]:
carDF = checkRegex('rule4_10',carDF,'year','\d+$')

In [62]:
carDF.show()

+------+---------+-------+--------+
|car_id|car_model|rule4_1|rule4_10|
+------+---------+-------+--------+
|     1|     XC40|  false|    true|
|     2|     XC60|   true|   false|
|     3|      S90|   true|    true|
|     4|      S60|   true|    true|
|     5|    P1800|   true|    true|
|     6|       TT|   true|    true|
|     7|       A3|   true|    true|
|     8|       A4|   true|    true|
|     9|       A6|   true|    true|
|    10|      330|   true|    true|
|    11|      340|   true|    true|
|    12|      540|   true|    true|
|    13|       M3|   true|    true|
|    14|       XX|   true|    true|
+------+---------+-------+--------+



#### Rule 4_2

In [27]:
case4_2 = carDF.year.rlike('\d+$')

In [28]:
carDF = carDF.withColumn('rule4_2', case4_2)

### Rule 5

#### Rule 5_1

In [29]:
split5_1 = split(customer_locationDF.chg_dt,'\W').alias('new_chg_dt')

In [50]:
# Define if the value is 4 numerical digits
def isYear(arg):
    if re.match('\d\d\d\d$', arg):
        return True
    else:
        return False

# Define if the value is a number in a 1-31 range    
def isDay(arg):    
    if re.match('^\d{1,2}$', arg):
        if int(arg)>0 and int(arg)<=31:
            return True
        else:
            return False
    else:
        return False

# Define if the value is considerable as a month and return it in a numerical format
def isMonth(arg):
    months = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']
    
    if re.match('^\d{1,2}$', arg):
        if int(arg)>0 and int(arg)<=12:
            return arg
        else:
            return arg
    elif any(re.match('^'+arg.upper(),month) for month in months):
        month = next(i+1 for i,month in enumerate(months) if re.match('^'+arg.upper(),month))
        return "{0:0=2d}".format(month)
    else:
        return False

In [55]:
def getDate(row):    
    if isYear(row[0]) and isMonth(row[1]) and isDay(row[2]):
        return row[0]+'-'+str(isMonth(row[1]))+'-'+row[2]
        
    elif isDay(row[0]) and isMonth(row[1]) and isYear(row[2]):
        return row[2]+'-'+str(isMonth(row[1]))+'-'+row[0]
        
    elif isMonth(row[0]) and isDay(row[1]) and isYear(row[2]):
        return row[2]+'-'+str(isMonth(row[0]))+'-'+row[1]
        
    else:
        return None
getDateUDF = udf(getDate)

In [52]:
def transformDateFormat(name,df,column):
    case = split(column,'\W').alias(name)
    
    result = df.withColumn(name, getDateUDF(case))
    
    return result   

In [32]:
case5_1 = customer_locationDF.join(timeDF.select('time_dt'), getDateUDF(split5_1) == timeDF.time_dt, 'left')

In [33]:
customer_locationDF = case5_1.withColumn('rule5_1', when(case5_1.time_dt.isNull(),False).otherwise(True)).drop('time_dt')

In [56]:
customer_locationDF = transformDateFormat('new_date',customer_locationDF,customer_locationDF.chg_dt)

In [59]:
customer_locationDF = checkLink('rule5_10',customer_locationDF,customer_locationDF.new_date,timeDF,'time_dt')

In [60]:
customer_locationDF.show()

+-------+------+-----------+----------+--------+
|Cust_id|loc_id|     chg_dt|  new_date|rule5_10|
+-------+------+-----------+----------+--------+
|      1|     1| 2019-01-21|2019-01-21|    true|
|      2|     2| 2020-01-19|2020-01-19|    true|
|      3|     3| 2019-02-27|2019-02-27|    true|
|      4|     4| 2019-09-26|2019-09-26|    true|
|      5|     5| 2019-05-07|2019-05-07|    true|
|      6|     6| 2019-05-17|2019-05-17|    true|
|      7|     7| 2020-03-26|2020-03-26|    true|
|      8|     8| 2019-10-01|2019-10-01|    true|
|      9|     9| 2019-10-03|2019-10-03|    true|
|     10|    10| 2020-01-12|2020-01-12|    true|
|     11|    11| 2019-06-19|2019-06-19|    true|
|     12|    12| 2019-07-27|2019-07-27|    true|
|     13|    13| 2019-05-03|2019-05-03|    true|
|     14|    14| 2019-05-31|2019-05-31|    true|
|     15|    15| 2020-03-22|2020-03-22|    true|
|     16|    16|2020 Jan 12|2020-01-12|    true|
|     17|    17|2019-Rea-10|      null|   false|
|     18|    18| 201

#### Rule 5_2

In [34]:
temp = customerDF.withColumnRenamed('cust_id','temp').select('temp')

In [35]:
case5_2 = customer_locationDF.join(temp, customer_locationDF.Cust_id == temp.temp, how='left')

In [36]:
customer_locationDF = case5_2.withColumn('rule5_2', when(case5_2.temp.isNull(),False).otherwise(True)).drop('temp')

#### Rule 5_3

In [37]:
temp = locationDF.withColumnRenamed('loc_id', 'temp').select('temp')

In [38]:
case5_3 = customer_locationDF.join(temp, customer_locationDF.loc_id == temp.temp, 'left')

In [39]:
customer_locationDF = case5_3.withColumn('rule5_3', when(case5_3.temp.isNull(),False).otherwise(True)).drop('temp')

In [42]:
customer_locationDF.filter(customer_locationDF.rule5_3==False).show()

+-------+------+------+-------+-------+-------+
|Cust_id|loc_id|chg_dt|rule5_1|rule5_2|rule5_3|
+-------+------+------+-------+-------+-------+
+-------+------+------+-------+-------+-------+



In [48]:
locationDF.

TypeError: filter() got an unexpected keyword argument 'column'